<a href="https://colab.research.google.com/github/pragmalingu/experiments/blob/master/Data/MedlineCorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup an Elasticsearch Instance in Google Colab

Everthing to connect to Elasticsearch, for detailed explaination see [this Notebook.](https://)
Download:

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
# download elasticsearch
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.1-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.1-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.1

Start a local server:

In [ ]:
# start server
es_server = Popen(['elasticsearch-7.9.1/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# client-side
!pip install elasticsearch -q
from elasticsearch import Elasticsearch
from datetime import datetime
es = Elasticsearch()
es.ping()  # got True

# Parsing Medline Corpus

You can get the corpus from [this link](http://ir.dcs.gla.ac.uk/resources/test_collections/medl/).  <br>
For detailed information about the format of the files, see the PragmaLingu [ Benchmarks](https://pragmalingu.de/docs/benchmarks/overview)

### Dowlnoad Data

Download and unzip data.

In [ ]:
!wget http://ir.dcs.gla.ac.uk/resources/test_collections/medl/med.tar.gz
!tar -xf med.tar.gz

Set paths to the dowloaded data as variables:

In [ ]:
PATH_TO_MED_TXT = '/content/MED.ALL'
PATH_TO_MED_QRY = '/content/MED.QRY'
PATH_TO_MED_REL = '/content/MED.REL'

### Imports

Make all the imports:

In [ ]:
from collections import defaultdict
import re
import json
from io import StringIO
import numpy as np

### Process Data

Get the text entries from the text and query file preprocessed as a list:

In [ ]:
ID_marker = re.compile('^\.I',re.MULTILINE)

def get_data(PATH_TO_FILE, marker):
  """
  Reads file and spilts text into entries at the ID marker '.I'.
  First entry is empty, so it's removed.
  'marker' contains the regex at which we want to split
  """
  with open (PATH_TO_FILE,'r') as f:
    text = f.read()
    lines = re.split(marker,text)
    lines.pop(0)
  return lines

med_txt_list = get_data(PATH_TO_MED_TXT, ID_marker)
med_qry_list = get_data(PATH_TO_MED_QRY, ID_marker)

Process the list of the texts and queries into nested dictionaries which can be saved as json:

In [ ]:
qry_chunk_start = re.compile('\.W')

# process the document data

med_txt_data = defaultdict(dict)
med_qry_data = defaultdict(dict)

def fill_dictionary(dictionary, chunk_list, marker, key_name):
  for n in range(0,len(chunk_list)-1):
    line = chunk_list[n+1]
    _ , chunk = re.split(marker,line)
    dictionary[n+1][key_name] = chunk.strip()

fill_dictionary(med_txt_data, med_txt_list, qry_chunk_start, 'text')
fill_dictionary(med_qry_data, med_qry_list, qry_chunk_start, 'question')

Relevance assesments are saved as numpy and parsed to dictionary:

In [ ]:
# process relevance assesments with rating

med_rel_data = open(PATH_TO_MED_REL)
med_np = np.loadtxt(med_rel_data, dtype=int)

med_rel_rat = defaultdict(list)
for row in med_np:
  med_rel_rat[row[0]].append(row[2])

# process relevance assesments without rating
med_rel = defaultdict(list)

with open (PATH_TO_MED_REL,'r') as f:
  for line in f:
    line = re.split(' ',line)
    med_rel[int(line[0])].append(line[2])

### Create index for Medline corpus

Create an index for the medline corpus. This is only possible if it isn't created yet.

(For more information see the [Elasticsearch documentation](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.create))

In [ ]:
#create index, see https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.create
es.indices.create("medline-corpus")
#print new index list
create_response = es.cat.indices()
print(create_response)

Index all the documents that are processed to the created index in elasticsearch:

(For more information see the [Elasticsearch documentation](https://elasticsearch-py.readthedocs.io/en/master/#example-usage))

In [ ]:
#index document, see https://elasticsearch-py.readthedocs.io/en/master/#example-usage
med_index = "medline-corpus"

for ID, doc_data in txt_data.items():
  es.index(index=med_index, id=ID, body=doc_data)


Verify if everthing went right by printing it:

In [ ]:
create_response = es.cat.indices()
print(create_response)

yellow open security-auditlog-2020.07.26 3KQWvUwlRHOYj3AeOYcfkg 1 1   31 0  75.6kb  75.6kb
yellow open security-auditlog-2020.07.27 C3A4S_lhSC-y5NEvhhhAAg 1 1   20 0  36.9kb  36.9kb
yellow open security-auditlog-2020.07.28 sLtcImDYTmafKHDoxhmg8g 1 1   22 0  72.3kb  72.3kb
yellow open cranfield-corpus             _d6ZOeRPRGOnb2ftarkP3Q 1 1 1400 0   1.6mb   1.6mb
green  open .kibana_92668751_admin_1     VkJe6jjiSMSj77PJCniIZQ 1 0    1 0   3.8kb   3.8kb
yellow open pragmalingu-test             f_mKvLYwT9uqrczeykbXxw 1 1    0 0    208b    208b
yellow open security-auditlog-2020.07.29 s_wFvB3PTeKWfVSZFdlgOw 1 1    9 0 123.3kb 123.3kb
green  open stemmer-cranfield-corpus     sNr-pythTYWzdQtzkPDuAw 1 0 1400 0   1.7mb   1.7mb
green  open .kibana_92668751_admin_2     jqG35SnPSluE8FaWoIYZmg 1 0    1 0   3.8kb   3.8kb
yellow open adi-corpus                   GcMzBe2bTC28dNE6-UsucQ 1 1   82 0 130.5kb 130.5kb
green  open .kibana_1                    JQnZPkFUSmO1fr5WE0TSzw 1 0    0 0    208b    208b

### Use Corpus in Ranking API

Use the ranking evaluation API from elasticsearch to evaluate the corpus:

(For more information see the [python documentation](https://elasticsearch-py.readthedocs.io/en/master/api.html?highlight=_rank_eval#elasticsearch.Elasticsearch.rank_eval) and the [Elasticsearch documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-rank-eval.html#search-rank-eval))

In [ ]:
#use rank eval api, see https://elasticsearch-py.readthedocs.io/en/master/api.html?highlight=_rank_eval#elasticsearch.Elasticsearch.rank_eval 
#and https://www.elastic.co/guide/en/elasticsearch/reference/current/search-rank-eval.html#search-rank-eval
from collections import defaultdict

med_index = "medline-corpus"

def create_query_body(query_dict, rel_dict, index_name):
  """
  The function creates a request for every query in query_dict and rates the relevant documents with rel_dict to 1.
  The index name has to be the same as from the documents your looking at.
  An evaluation body for the elasticsearch ranking API is returned.
  """
  eval_body = {
      "requests":'',
      "metric": {
          "recall": {
              "relevant_rating_threshold": 1,
              "k": 20
              }
      }
  }
  requests = [] 
  current_request = defaultdict(lambda: defaultdict())
  current_rel = {"_index": index_name, "_id": '', "rating": int}
  for query_ID, query_txt in query_dict.items():
    current_query = {"query": { "multi_match": { "query": '' , "fields" : ["title","text"]}}}
    current_query["query"]["multi_match"]["query"] = query_txt['question']
    current_request["id"] = 'Query_'+str(query_ID)
    current_request["request"] = current_query.copy()
    current_request["ratings"] = [{"_index": index_name, "_id": str(el), "rating": 1} for el in rel_dict[query_ID]]
    requests.append(current_request.copy())
  eval_body["requests"] = requests
  return eval_body

med_create = create_query_body(med_qry_data, med_rel, med_index)

med_eval_body = json.dumps(med_create)

Print results of Ranking API:

In [ ]:
med_res = es.rank_eval(med_eval_body, med_index)

print(json.dumps(med_res, indent=4, sort_keys=True))